In [ ]:
import csv
import time
from datetime import datetime

# ta no diretorio 
import pagarme as pm
import horario

In [ ]:
with open("key/key.txt", "r") as _key:
    pm.authentication_key(_key.read())

transaction = pm.transaction.find_all()

In [ ]:
count_t = 0
for tran_teste in transaction:
    count_t += 1
print("Número de transações: {}".format(count_t))
print("Tamanho com len(): {}".format(len(transaction)))

In [ ]:
# ALGUMAS REQUISIÇÕES DE TESTE
transacao = pm.transaction.find_by({"count":"300"})
cont_bili = 0
cont_shi = 0
cont_adr = 0
cont_item = 0
for tr in transacao:
    if len(tr['items']) > 0:
        cont_item += 1
    if tr['address'] == None:
        pass
    else:
        if len(tr['address']) > 0:
            cont_adr += 1
    if tr['billing'] == None:
        pass
    else:
        if len(tr['billing']) > 0:
            cont_bili += 1
    if tr['shipping'] == None:
        pass
    else:
        if len(tr['shipping']) > 0:
            cont_shi += 1

print("Transacoes com Endereço  | {}".format(cont_adr))
print("    ||     com Billing   | {}".format(cont_bili))
print("    ||     com Shipping  | {}".format(cont_shi))
print("    ||     com items     | {}".format(cont_item))

# Classes e Funções

In [ ]:
from datetime import datetime
import sys
#import time
data_atual = horario.data_atual
ano_atual = horario.ano_atual

class Transacao:
    def __init__(self,transacao):
        self.objeto = transacao['object'] # string
        self.id = transacao['id'] # Integer
        self.id_assinatura = transacao['subscription_id'] # integer
        self.tid = transacao['tid'] # Integer
        self.status = transacao['status'] # string
        self.usuario = transacao['customer'] # dict | objeto usuario
        self.email = self.usuario['email'] # string 
        self.data_criacao = transacao['date_created'] # str
        self.data_update = transacao['date_updated'] # str
        self.valor = transacao['amount'] / 100 # Integer, vira float
        self.valor_captado = transacao['paid_amount'] / 100 # Integer, vira float
        self.valor_estornado = transacao['refunded_amount'] / 100 # Integer, vira float
        self.parcelas = transacao['installments'] # Integer
        self.cobranca = transacao['billing'] # dict | objeto cobranca
        self.endereco = transacao['address'] # dict | objeto endereço
        self.completo = transacao # dict | objeto transacao
    
    def get_objeto(self):
        """
        Checa se é uma transacao mesmo, ele devolve o 'object' dela como transaction.
        """
        if self.objeto.lower() != "transaction":
            print(f"{self.objeto}, não é uma transacao, cuidado em tio, f")
            return False
        else:
            print("Chamou o var e confirmou, segue o jogo, é transação!")
            return True
    
    # FUNCOES DE INFORMAÇÕES BASICAS
    def get_info_basica(self):
        """
        Retorna uma tupla, com as informações basicas da transacao, como a data(updated), valor e status.
        """
        # para devolver o valor em string normal no formato yyyy-mm-dd
        data_simples = datetime.strptime(self.data_update,"%Y-%m-%dT%H:%M:%S.%fZ")
        data_simples = f"{data_simples.year}-{data_simples.month}-{data_simples.day}"
        return self.data_update, self.valor, self.status, self.id, self.email
    
    def get_valores(self):
        """ 
        Retorna um dicionario com os valores. Valor Pago, Valor Capturado e Valor Estornado.
        """
        _dicionario_valores = {"valor_brl" : self.valor,
                               "valor_captado" : self.valor_captado,
                               "valor_estornado" : self.valor_estornado}
        return _dicionario_valores
    
    def get_data_update(self):
        """
        Passamos a data que está em formato de string e datetime, para unix timestamp.
        """
        _data = self.data_update
        _data = datetime.strptime(_data,"%Y-%m-%dT%H:%M:%S.%fZ")
            # tiramos 10800 pois o pagar.me devolve na hora UTC 0
            # (timestamp(_data) - 10800)
        _data = (datetime.timestamp(_data) - 10800) * 1000
        return _data
    
    def get_data_created(self):
        """
        Passamos a data que está em formato de string e datetime, para unix timestamp.
        """
        _data = self.data_criacao
        _data = datetime.strptime(_data,"%Y-%m-%dT%H:%M:%S.%fZ")
            # tiramos 10800 pois o pagar.me devolve na hora UTC 0
            # (timestamp(_data) - 10800)
        _data = (datetime.timestamp(_data) - 10800) * 1000
        return _data
    
    def get_endereco(self):
        """
        Retorna uma tupla com (cidade, estado, pais e CEP).
        """
        if self.endereco == None: # se n tiver nada no endereco, busca em cobrancas
            if self.cobranca != None: # se até o objeto de cobrança tiver limpo, passa vazio
                # nao vamos utilizar o endereço de entrega como parametro para endereço
                _cidade = self.cobranca['address']['city']
                _estado = self.cobranca['address']['state']
                _pais = self.cobranca['address']['country']
                _cep = self.cobranca['address']['zipcode']
            else:
                _cidade = " "
                _estado = " "
                _pais = " "
                _cep = " "
        else:    
            _cidade = self.endereco['city']
            _estado = self.endereco['state']
            _pais = self.endereco['country']
            _cep = self.endereco['zipcode']
        return _cidade,_estado,_pais,_cep
    
    
    
    # funcao para criar um dicionario e utilizar o pandas para analisar
    def get_dicionario_transacao(self):
        _cidade, _estado, *resto = self.get_endereco()
        
        _dicionario_transacao = {
                    "status": self.status, # Status
                    "id": str(self.id), # ID
                    "subscription_id": str(self.id_assinatura),  # ID da Assinatura
                    "nome": self.usuario['name'], # Nome
                    "email": self.email, # Email
                    "valor_brl": self.valor, # Valor (R$)
                    "valor_captura_brl": self.valor_captado, # Valor Capturado (R$
                    "valor_estornado_brl": self.valor_estornado, # Valor Estornado (R$)
                    "num_parcelas": self.parcelas, # Número de Parcelas
                    "TID" : str(self.tid), # TID
                    "cidade": _cidade, # Cidade
                    "estado": _estado, # Estado
                    "data_criacao": self.data_criacao, # Data
                    "data_atualizacao": self.data_update, # Última Atualização
                    }
        return _dicionario_transacao
        
        
    # para criar lista de público para o facebook
    def get_dados_facebook(self):
        email = self.usuario['email']
        telefone = self.usuario['phone_numbers']
        
        if type(telefone) == list:
            telefone = telefone[0]

        nome = self.usuario['name'].split()
        n_nome = len(nome)
        if n_nome == 1:
            fn = nome[0]
            ln = " "
        elif 0 >= n_nome:
            fn = " "
            ln = " "
        else:
            fn = nome[0]
            ln = " ".join(nome[1:]) #[_ for _ in nome[1:]]

        # dados do endereço
            # chamando o método de endereço e passando em cada var referente
        cidade, estado, pais, cep = self.get_endereco()

        # yyyy-mm-dd
        data_nasci = self.usuario['birthday']
        if data_nasci == None:
            ano_nasci = " "
            idade = " "
            data_nasci = " "
        else:
            data_nasci_datetime = datetime.strptime(data_nasci,"%Y-%m-%d")
            ano_nasci = data_nasci_datetime.year
            if ano_nasci > (ano_atual - 5):
                ano_nasci = " "
                idade = " "
                data_nasci = " "
            else:
                idade = ano_atual - ano_nasci

        return fn,ln,email,telefone,cidade,estado,pais,data_nasci,ano_nasci,idade
    
    
# FUNCOES DE FORA DA CLASSE
def ultima_transacao(transacoes):
    """Retorna a ultima transação das transações."""
    return transacoes[-1]

In [ ]:
# Para instanciar a classe, basta chama-la armazenando em uma variavel, 
test_classe = Transacao(tran_teste)
# nesse caso estamos deixando a transacao dentro do objeto com nome 'test_classe'

# podemos chamar o atributo .objeto para checar qual é
test_classe.objeto # -> 'transaction'

print(test_classe.get_dados_facebook())

t_teste = test_classe.get_dicionario_transacao()

# Fazendo a chamada na API do pagar.me, utilizando da classe Transacao para analise das transacões

In [ ]:
# quanto tempo durou a requisicao - começo
tempo_agora = time.time()

# --> variaveis de contagem/soma
soma_valor_brl = 0
soma_valor_capturado = 0
soma_valor_estornado = 0

count_transacoes = 0
count_requisicoes = 0

LISTA_VALORES = list()
LISTA_DATAS = list()


# a hora vem em segundos, precisamos passar em milisegundos
data_parametro = time.time() * 1000 
print(" --- DATA PARAMETRO: {}".format(data_parametro))
# declarações para a saida de loading
print("É para puxar até qual data:")


_data = horario.get_string_data() # funcao para puxar um input e inserir as datas
por_inicial = data_parametro # data inicial, tbm a porcentagem inicial
data_final = horario.get_unixtime_data(_data) # data final e tbm a porcentagem final
por_dif = data_final - por_inicial # a diferença entra a data de inicio e fim
# por_atual = (atual - por_inicial) / por_dif

# DECLARAÇÕES IMPORTANTES | CSV
LISTA_FB_CSV = list()
LISTA_DATAFRAME = list()

# declaracao para o while
tem_transacao = True
while tem_transacao:
    transacoes = pm.transaction.find_by({"count": "500",
                                         "date_created": f"<{data_parametro}",
                                         "status": "paid"})
    
    # se retornar nenhuma transacao em transacoes, pode parar tbm
    count_requisicoes += 1
    if len(transacoes) == 0:
        tem_transacao = False
        break
            
    for ite_transacao in transacoes:
        # instanciando a classe e gerando o objeto ref a transação
        objeto_transacao = Transacao(ite_transacao)
    
        # declarando a variavel para manter a data de criacao da transacao
        data_parametro = objeto_transacao.get_data_created()
        if data_parametro <= data_final - 10800: # DATA FINAL
            # se a data de criacao for menor que a data que selecionamos, pare o loop
            tem_transacao = False
            break
        # contagem das transacoes
        count_transacoes += 1

        # declaracoes para variaveis de contagem/soma
        _valores_transacao = objeto_transacao.get_valores()
        LISTA_VALORES.append(_valores_transacao)
        LISTA_DATAS.append(objeto_transacao.data_criacao)

        # variaveis para observar valores do periodo selecionado
        soma_valor_brl += _valores_transacao['valor_brl']
        soma_valor_capturado += _valores_transacao['valor_captado']
        soma_valor_estornado += _valores_transacao['valor_estornado']

        # passando as transacoes para uma lista
        LISTA_FB_CSV.append(list(objeto_transacao.get_dados_facebook()))
        LISTA_DATAFRAME.append(objeto_transacao.get_dicionario_transacao())
            
            
    # passamos novamente a data de criacao para a variavel data parametro
    # que vai ser usada na nova requisicao
    data_parametro = objeto_transacao.get_data_created()
    
    # saindo para a tela de carregamento
    por_atual = (data_parametro - por_inicial) / por_dif
    sys.stdout.write("\r" + "CARREGANDO, PERA AI... {:.1f} %".format(por_atual * 100))
    sys.stdout.flush()
    
    # limite de 1.000 por minuto, então da um tempo para fazer a proxima
    time.sleep(30)
    
    
# quanto tempo durou a requisicao
tempo_depois = time.time()
# agora mostra os valores e umas informações sobre as chamadas
print()
print("-------- VALORES --------")
print("|- Valor Total Pago: R$ {:,}".format(soma_valor_brl))
print("|- Valor Total Capturado: R$ {:,}".format(soma_valor_capturado))
print("|- Valor Total Estornado: R$ {:,}".format(soma_valor_estornado))
print("-------------------------------")
print("Soma count transacoes: {}".format(count_transacoes))
print("Soma count requisições: {}".format(count_requisicoes))
print("------------------------")
print("Data da transacao do BREAK: {}".format(objeto_transacao.data_update))
print("Data da ultima transacao: {}".format(LISTA_DATAFRAME[-1]['data_criacao']))
print("------------------------")
print("Requisicao durou: {}".format(horario.get_standard_format(tempo_depois - tempo_agora)))
# 1640995200000 -> timestamp janeiro 01/01/2022 00:00:00 | GTM TIME +0

In [ ]:
print("Data das 5 primeiras transações")
for n,i in enumerate(LISTA_DATAFRAME[:5]):
    print("{} | {}".format(n,i['data_criacao']))

print("Data das 5 ultimas transações")
for n,i in enumerate(LISTA_DATAFRAME[-5:]):
    print("{} | {}".format(n,i['data_criacao']))

# Passando para o .csv para pandas | DATAFRAME

In [ ]:
# declaracao para lista do DataFrame é:
LISTA_DATAFRAME

with open("dados/dataset_{}_{:02d}_{:02d}.csv".format(data_atual.year,
                                                      data_atual.month,
                                                      data_atual.day),
          "w", newline = "") as file:
    
    writer = csv.writer(file,delimiter=",")
    writer.writerow(LISTA_DATAFRAME[1].keys())
    for ite_row in LISTA_DATAFRAME:
        writer.writerow(ite_row.values())

## Criando .csv da INFO BASICA

In [ ]:
lista_numero = list()

for i in LISTA_FB_CSV:
    lista_numero.append(i[3])

lista_numero = list(dict.fromkeys(lista_numero))

with open ("dados/lista_pagar_me_numero.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile,delimiter=',')
    writer.writerow("numero_pagar_me")
    for i in lista_numero:
        writer.writerow(i)

## Montando a lista para o público do facebook direto com a requisição acima

In [ ]:
with open("dados/publicos/transacoes_basica.csv","w",newline="") as csvfile:
    writer = csv.writer(csvfile,delimiter=",")
    writer.writerow(["1_nome","2_nome","email","numero","city","st","ct","dob","y","age"])
    for ite_linha in LISTA_FB_CSV:
        writer.writerow(ite_linha)

## Criando o arquivo CSV com a lista que criamos 'LISTA_CSV', informações basicas USUARIOS

In [ ]:
with open("dados/transacoes.csv","w",newline="") as csvfile:
    writer = csv.writer(csvfile,delimiter=",")
    writer.writerow(["data","valor","status","id","email"])
    for ite_linha in LISTA_CSV:
        writer.writerow(ite_linha)

# Criação do .csv dos compradores do pagar.me | lista publico facebook. Fazendo a requisição

In [ ]:
## codigo para criar LISTA PÚBLICO FACEBOOK

# fn,ln,email,telefone,cidade,estado,pais,data_nasci,ano_nasci,idade
with open(f"dados/publicos/lista_publico_{data_atual.month}_{data_atual.day}_.csv","w",newline='') as csvfile:
    data_parametro = time.time() * 1000
    writer = csv.writer(csvfile, delimiter=",")
    # Passando os nomes da coluna (header)
    writer.writerow(["fn","ln","email","phone","ct","st","country","dob","doby","age"])
    while True:
        transacoes = pm.transaction.find_by({"count":"500","status":"paid","date_updated":f"<{data_parametro}"})
        if len(transacoes) == 0:
            break
        for ite_tran in transacoes:
            t = Transacao(ite_tran)
            data_parametro = t.get_data_update()
            t_fb = t.get_dados_facebook()
            writer.writerow([t_fb[0],t_fb[1],t_fb[2],t_fb[3],t_fb[4],t_fb[5],t_fb[6],t_fb[7],t_fb[8],t_fb[9]])
            
        time.sleep(40)

## LOUCURA PRA CA DE BAIXO

In [ ]:
# tentando fazer meio que uma paginação, pegar a ultima transação enviada, e fazer outra requisição a partir daquela data

transacoes = pm.transaction.find_by({'count':'8',"date_updated":"<1624367191178.0"})
print("Numero de transacoes: {}".format(len(transacoes)))
# checando as transacoes
for i,primeira in enumerate(transacoes):
    ult_tran = Transacao(primeira)
    data_ultima = ult_tran.get_data_update()
    print("{} | {}".format(i,ult_tran.data_update))
    print("{} | {}".format(i,ult_tran.get_data_update()))
    print()
    
# ult_tran = ultima_transacao(transacoes)

print("data da ultima transacao: {}".format(data_ultima))
#ult_tran = Transacao(ult_tran)
#data_ultima = ult_tran.get_data_update()
print("get_date_updated: {}".format(data_ultima))
print("---------------")

                                                # para puxar a ultima tbm, basta deixar um <=, ao inves de somente <
search_params = {"count":"7","date_updated":f"< {data_ultima}"}
pos_transacoes = pm.transaction.find_by(search_params)
print("Numero de transacoes: {}".format(len(pos_transacoes)))
for n,i in enumerate(pos_transacoes):
    ult_tran = Transacao(i)
    data_ultima = ult_tran.get_data_update()
    print("{} | {}".format(n,i['date_updated']))
    print("{} | {}".format(n,ult_tran.get_data_update()))
    print()


print("--------------------")
search_params = {"count":"5","date_updated":f"< {data_ultima}"}
pos_transacoes = pm.transaction.find_by(search_params)
print("Numero de transacoes: {}".format(len(pos_transacoes)))
for n,i in enumerate(pos_transacoes):
    ult_tran = Transacao(i)
    data_ultima = ult_tran.get_data_update()
    print("{} | {}".format(n,ult_tran.data_update))
    print("{} | {}".format(n,ult_tran.get_data_update()))
    print()

In [ ]:
#tr = pm.transaction.find_by({"count":"1","date_created":"DATE_CRATED"})
#tr
# print()
tr = pm.transaction.find_by({"id":"444673976"})
for i in tr:
    for j in i:
        print(f"{j:25} | {i[j]}")
        pass

# horario da transacao acima
print(i['date_created'])
import horario
hora = horario.get_unixtime_datetime(i['date_created'])
hora - (10800 * 1000)

In [ ]:
t = pm.transaction.find_by({"date_created":"<=1635732285250.0"})
for i in t:
    print(i['date_created'])
    print()

In [ ]:
data_pam = time.time() * 1000
datetime.fromtimestamp(time.time())

for i in range(10):
    tra = pm.transaction.find_by({"count":"1","date_created": f"<{data_pam}"})
    for t in tra:
        obj = Transacao(t)
        data_pam = obj.get_data_created()
        print(i)
        print(obj.data_criacao)
        print("Hora unixtime: {}".format(obj.get_data_created()))
        print("Data Parametr: {}".format(data_pam))
        print("-ID Transacao: {}".format(obj.id))
        print()
